# Άσκηση 1, Ανίχνευση ακμών:
Θεωρήστε  τις  εικόνες  αποχρώσεων  του  γκρι  (grayscale image) `leaf.jpg`, `X-ray.jpeg` και `parking-lot.jpg`. 
 
**Ζητούμενα:** 

Α. Σχεδιάστε, περιγράψτε και εφαρμόστε 3 διαφορετικές μεθόδους για την ανίχνευση των κύριων ακμών κάθε εικόνας (δηλαδή να δοθεί έμφαση στην ανίχνευση των ορίων και των περιγραμμάτων των εικονιζόμενων αντικειμένων). 

Β. Για κάθε εικόνα, τυπώστε την αρχική εκδοχή της και τις 3 βέλτιστες δυαδικές μάσκες ανίχνευσης ακμών κάθε μεθόδου. Σχολιάστε τα αποτελέσματα. 

Για όλες τις περιπτώσεις ανίχνευσης ακμών, τυπώστε μόνο τη (βέλτιστη) δυαδική μάσκα που προκύπτει από κάθε μέθοδο και εξηγήστε λεπτομερώς τη διαδικασία σχεδίασης (π.χ. επιλογή μεθόδου, καθορισμός παραμέτρων, κτλ.).

In [ ]:
%pip install matplotlib numpy scikit-image scikit-learn --quiet

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

# Φόρτωση της εικόνας
image_flowers = ski.io.imread("https://github.com/KaratziasK/Digital-Image-Processing/blob/main/project-1/instructions/images-project-1/flowers.jpg?raw=true")
